In [49]:
# coding:utf-8
"""
本篇为单独调试destine()函数的调试记录，后期将加入UserMainData
"""
import pandas as pd
import numpy as np
from impala.dbapi import connect
from impala.util import as_pandas
import cPickle
import re
import datetime as dt
import pymysql.cursors

# Global variables
mysql_host = '172.21.1.151'
mysql_port = 3306
mysql_user = 'xli'
mysql_password = '123456'
mysql_db = 'kdwtemp2'
mysql_charset = 'utf8mb4'

def easy_print(terms,tname, records=10): #逐条描述前五条记录，方便仔细查看表中值的特征
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT ' + terms + ' FROM ' + tname + ' LIMIT 5' 
    cur.execute(sql)
    read_in = as_pandas(cur)    
    cur.close()
    conn.close()
    
    #print in a format
    columns = read_in.columns.values
    values = read_in.values
    records = []
    for v in values:
        records.append(zip(columns, v))
    
    for ix,r in enumerate(records):
        print('----------第%s条记录--------' % (ix+1))
        for k, v in r:
            print('%s: %s' % (k, v))
            
def full_print(terms,tname): #打出全表所选列，以方便进行统计查看规律，决定处理方法
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT ' + terms + ' FROM ' + tname
    cur.execute(sql)
    df = as_pandas(cur)
    print df
    cur.close()
    conn.close()
    
def str_to_float(data):
    return float(data)

def convert_date_weekday(data):
    data = pd.to_datetime(data, format='%Y-%m-%d')
    return data.weekday()
    
def convert_hour(data):
    data = pd.to_datetime(data, format='%Y-%m-%d %H:%M:%S')
    return data.hour

def convert_time(data):
    data = pd.to_datetime(data, format='%Y-%m-%d %H:%M:%S')
    return data

def convert_time_hour(data):
    return data.hour

def convert_timedelta_hour(data):
    if data.days > 0:
        return (data.days*24 + data.seconds/3600)
    else:
        return data.seconds/3600

def convert_date_time(data):
    data = pd.to_datetime(data, format='%Y-%m-%d')
    return data

def is_large(data):
    pattern = re.compile(r'大包')
    if re.search(pattern,data):
        return 1
    else:
        return 0
    
def is_medium(data):
    pattern = re.compile(r'中包')
    if re.search(pattern,data):
        return 1
    else:
        return 0
    
def is_small(data):
    pattern = re.compile(r'小包')
    if re.search(pattern,data):
        return 1
    else:
        return 0

def is_reduce(data):
    pattern = re.compile(r'免包厢费')
    if re.search(pattern,data):
        return 1
    else:
        return 0
def des_before_dawn(data):
    if data < 6:
        return 1
    else:
        return 0
    
def des_morning(data):
    if data >=6 and data < 12:
        return 1
    else:
        return 0
def des_afternoon(data):
    if data >= 12 and data < 18:
        return 1
    else:
        return 0
    
def des_night(data):
    if data >= 18:
        return 1
    else:
        return 0    
def destine(): #包间消费信息统计
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()    
    
    # 1. 包间消费记录数据统计
    sql = 'SELECT customer_id as customerid,guest_number,destine_room_charge,protocol_room_charge,reduce_cost,used_date FROM km_tbl_des_destine_km'
    cur.execute(sql)
    df = as_pandas(cur)
    df['des_cnt'] = 1
    df['guest_number'] = map(str_to_float,df['guest_number'])
    df['destine_room_charge'] = map(str_to_float,df['destine_room_charge'])
    df['protocol_room_charge'] = map(str_to_float,df['protocol_room_charge'])
    df['reduce_cost'] = map(str_to_float,df['reduce_cost'])
    df['des_charge_sum'] = df['destine_room_charge']/df['guest_number']
    df['des_procharge_sum'] = df['protocol_room_charge']/df['guest_number']
    df['des_reduce_sum'] = df['reduce_cost']/df['guest_number']
    df['des_realcharge_sum'] = (df['destine_room_charge']-df['reduce_cost'])/df['guest_number']
    df['used_date'] = map(convert_date_weekday,df['used_date']) 
    df['des_workday_cnt'] = map(int,df['used_date'].isin({0,1,2,3,4}))
    df['des_weekend_cnt'] = map(int,df['used_date'].isin({5,6}))
    df['des_work_realchargesum'] = df['des_realcharge_sum']*df['des_workday_cnt']
    df['des_weekend_realchargesum'] = df['des_realcharge_sum']*df['des_weekend_cnt']
    df.drop('used_date',axis=1,inplace=True)
    # 求和
    df1 = df.groupby('customerid',as_index=False).sum()
    drop_columns = ['destine_room_charge','protocol_room_charge','reduce_cost']
    df1.drop(drop_columns,axis=1,inplace=True)
    # 求平均值
    df1['des_charge_mean'] = df1['des_charge_sum']/df1['des_cnt']
    df1['des_realcharge_mean'] = df1['des_realcharge_sum']/df1['des_cnt']
    df1['des_guestnumber_mean'] = df1['guest_number']/df1['des_cnt']
    df1.drop('guest_number',axis=1,inplace=True)
    
    # 求最大值和最小值
    drop_columns = [ 'destine_room_charge','protocol_room_charge', 'reduce_cost', 'des_cnt', 'des_procharge_sum', 
                    'des_reduce_sum','des_workday_cnt', 'des_weekend_cnt', 'des_work_realchargesum','des_weekend_realchargesum']
    df2 = df.drop(drop_columns,axis=1)
    # guest_number最大最小值
    df4 = df2.drop(['des_charge_sum','des_realcharge_sum'],axis=1)
    df4 = df4.sort_values(['customerid','guest_number'],ascending=[1,0])
    df3 = df4.drop_duplicates(subset='customerid')
    df3.columns = ['customerid','des_guestnumber_max']
    df4.drop_duplicates(subset='customerid',keep='last',inplace=True)
    df4.columns = ['customerid','des_guestnumber_min']    
    # des_charge_max,des_charge_min
    df6 = df2.drop(['guest_number','des_realcharge_sum'],axis=1)
    df6 = df6.sort_values(['customerid','des_charge_sum'],ascending=[1,0])
    df5 = df6.drop_duplicates(subset='customerid')
    df5.columns = ['customerid','des_charge_max']
    df6.drop_duplicates(subset='customerid',keep='last',inplace=True)
    df6.columns = ['customerid','des_charge_min']         
    # des_realcharge_max,des_real_charge_min
    df8 = df2.drop(['guest_number','des_charge_sum'],axis=1)
    df8 = df8.sort_values(['customerid','des_realcharge_sum'],ascending=[1,0])
    df7 = df8.drop_duplicates(subset='customerid')
    df7.columns = ['customerid','des_realcharge_max']
    df8.drop_duplicates(subset='customerid',keep='last',inplace=True)
    df8.columns = ['customerid','des_realcharge_min']    

    # merge所有中间表
    df = pd.merge(df3,df4,how='outer',left_on='customerid',right_on='customerid')
    df9 = pd.merge(df5,df6,how='outer',left_on='customerid',right_on='customerid')
    df10 = pd.merge(df7,df8,how='outer',left_on='customerid',right_on='customerid')
    df = pd.merge(df,df9,how='outer',left_on='customerid',right_on='customerid')
    df = pd.merge(df,df10,how='outer',left_on='customerid',right_on='customerid')
    df = pd.merge(df1,df,how='outer',left_on='customerid',right_on='customerid')
    des_stat = df

    # 2. 包厢类型统计
    sql = 'SELECT customer_id as customerid,buy_break_name FROM km_tbl_des_destine_km'
    cur.execute(sql)
    df = as_pandas(cur)    
    df['des_large_cnt'] = map(is_large,df['buy_break_name'])
    df['des_medium_cnt'] = map(is_medium,df['buy_break_name'])
    df['des_small_cnt'] = map(is_small,df['buy_break_name'])
    df['des_reduce_cnt'] = map(is_reduce,df['buy_break_name'])
    df.drop('buy_break_name',axis=1,inplace=True)
    des_type = df.groupby('customerid',as_index=False).sum()
    
    # 3. 时间相关统计操作
    sql = 'SELECT customer_id as customerid,destine_date_time,used_begin_time,used_end_time,destine_room_charge,reduce_cost,guest_number FROM km_tbl_des_destine_km'
    cur.execute(sql)
    df = as_pandas(cur)
    df['guest_number'] = map(str_to_float,df['guest_number'])
    df['destine_room_charge'] = map(str_to_float,df['destine_room_charge'])
    df['reduce_cost'] = map(str_to_float,df['reduce_cost'])
    df['des_realcharge_sum'] = (df['destine_room_charge']-df['reduce_cost'])/df['guest_number']
    df['destine_date_time'] = map(convert_hour,df['destine_date_time'])
    df['used_begin_time'] = map(convert_time,df['used_begin_time'])
    df['used_end_time'] = map(convert_time,df['used_end_time'])
    df['use_time_sum'] = map(convert_timedelta_hour,df['used_end_time'] - df['used_begin_time'])
    df['used_begin_time'] = map(convert_time_hour,df['used_begin_time'])
    df['used_end_time'] = map(convert_time_hour,df['used_end_time'])
    # 预定时间段计数
    df['des_beforedawn_cnt'] = map(des_before_dawn,df['destine_date_time'])
    df['des_morning_cnt'] = map(des_morning,df['destine_date_time'])
    df['des_afternoon_cnt'] = map(des_afternoon,df['destine_date_time'])
    df['des_night_cnt'] = map(des_night,df['destine_date_time'])
    df['des_use_beforedawn_cnt'] = map(des_before_dawn,df['used_begin_time'])
    df['des_use_morning_cnt'] = map(des_morning,df['used_begin_time'])
    df['des_use_afternoon_cnt'] = map(des_afternoon,df['used_begin_time'])
    df['des_use_night_cnt'] = map(des_night,df['used_begin_time'])    
    df['des_use_beforedawn_realchargesum'] = df['des_realcharge_sum']*df['des_use_beforedawn_cnt']
    df['des_use_morning_realchargesum'] = df['des_realcharge_sum']*df['des_use_morning_cnt']
    df['des_use_afternoon_realchargesum'] = df['des_realcharge_sum']*df['des_use_afternoon_cnt']
    df['des_use_night_realchargesum'] = df['des_realcharge_sum']*df['des_use_night_cnt']
    df = df.groupby('customerid',as_index=False).sum()
    drop_columns = ['destine_date_time', 'used_begin_time','used_end_time', 'destine_room_charge',
                    'reduce_cost','guest_number', 'des_realcharge_sum']
    df.drop(drop_columns,axis=1,inplace=True)
    # 求每个用户在线预订和使用包厢的平均时间差，单位为天
    sql = 'SELECT customer_id as customerid,used_begin_time,destine_date_time FROM km_tbl_des_destine_km'
    cur.execute(sql)
    df1 = as_pandas(cur)
    df1['used_begin_time'] = map(convert_time,df1['used_begin_time'])
    df1['destine_date_time'] = map(convert_time,df1['destine_date_time'])
    df1['des_use_reg'] = df1['used_begin_time'] - df1['destine_date_time']
    df1['des_use_reg'] = map(lambda x:x.days, df1['des_use_reg'])
    df1['des_cnt'] = 1
    df1 = df1.groupby('customerid',as_index=False).sum()
    df1['des_use_reg'] = df1['des_use_reg']/df1['des_cnt']
    df1.drop('des_cnt',axis=1,inplace=True)
    df = pd.merge(df,df1,how='left',left_on='customerid',right_on='customerid')
    des_time = df
    
    # 3. 统计消费所在商户数据
    # 求用户在每个商户消费次数
    sql = 'SELECT customer_id as customerid,company_id as des_companyid FROM km_tbl_des_destine_km'
    cur.execute(sql)
    df = as_pandas(cur)
    df['des_company_cnt'] = 1
    df = df.groupby(['customerid','des_companyid'],as_index=False).sum()
    # 保留消费次数最多的商户记录
    df = df.sort_values(['customerid','des_company_cnt'],ascending=[1,0])
    df.drop_duplicates(subset='customerid',inplace=True)
    des_com = df
    
    # 商户静态数据
    sql = """SELECT companyid as des_companyid,roomnum as des_com_roomnum,managetype as des_com_managetype,
    vodsystemtype as des_com_vodsystemtype,regioncode as des_com_regioncode FROM kdw_tbl_companybase"""
    cur.execute(sql)
    company = as_pandas(cur)
    
    # 4. 集合所有的表
    df = pd.merge(des_com,company,how='left',left_on='des_companyid',right_on='des_companyid')
    df = pd.merge(des_time,df,how='left',left_on='customerid',right_on='customerid')
    df = pd.merge(df,des_type,how='left',left_on='customerid',right_on='customerid')
    df = pd.merge(des_stat,df,how='left',left_on='customerid',right_on='customerid')
    
    cur.close()
    conn.close()
    # 将中间结果暂存于Mysqldb中
    conn = pymysql.connect(host=mysql_host,
                                      port=mysql_port,
                                      user=mysql_user,
                                      password=mysql_password,
                                      db=mysql_db,
                                      charset=mysql_charset)
    
    df.to_sql(name="ljl_destine", con=conn, flavor='mysql', if_exists='replace', index=False)
    conn.close()

In [50]:
destine()

In [19]:
df.head()

,customerid,des_cnt,des_charge_sum,des_procharge_sum,des_reduce_sum,des_realcharge_sum,des_workday_cnt,des_weekend_cnt,des_work_realchargesum,des_weekend_realchargesum,...,des_companyid,cnt,des_com_roomnum,des_com_managetype,des_com_vodsystemtype,des_com_regioncode,des_large_cnt,des_medium_cnt,des_small_cnt,des_reduce_cnt
0,0,2922,320684.000000,320684.000000,0.000000,320684.000000,1587,1335,187437.000000,133247.000,...,8158,292,72,1,0,500000,139,1105,1073,0
1,1008882,2,0.114286,0.114286,0.000000,0.114286,2,0,0.114286,0.000,...,10433,1,NaN,NaN,NaN,NaN,0,0,0,0
2,104173,6,0.038000,0.036000,0.000000,0.038000,6,0,0.038000,0.000,...,742,6,NaN,NaN,NaN,NaN,0,0,0,0
3,1043,4,30.866667,30.866667,4.833333,26.033333,4,0,26.033333,0.000,...,4018,2,NaN,NaN,NaN,NaN,0,0,0,0
4,105051,1,0.002000,0.002000,0.000000,0.002000,0,1,0.000000,0.002,...,742,1,NaN,NaN,NaN,NaN,0,0,0,0


In [20]:
df.shape

(1174, 43)

# 1. 对包间消费信息进行统计

### 初始读进来转换列数据类型结果

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4579 entries, 0 to 4578
Data columns (total 7 columns):
customerid              4579 non-null object
guest_number            4579 non-null float64
destine_room_charge     4579 non-null float64
protocol_room_charge    4579 non-null float64
reduce_cost             4579 non-null float64
des_cnt                 4579 non-null int64
des_charge              4579 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 286.2+ KB


In [17]:
df.shape

(4579, 6)

### df1 求和 求均值

In [219]:
df.head()

,customerid,des_cnt,des_charge_sum,des_procharge_sum,des_reduce_sum,des_realcharge_sum,des_workday_cnt,des_weekend_cnt,des_work_realchargesum,des_weekend_realchargesum,des_charge_mean,des_realcharge_mean,des_guestnumber_mean
0,0,2922,320684.000000,320684.000000,0.000000,320684.000000,1587,1335,187437.000000,133247.000,109.748118,109.748118,1.00
1,1008882,2,0.114286,0.114286,0.000000,0.114286,2,0,0.114286,0.000,0.057143,0.057143,4.00
2,104173,6,0.038000,0.036000,0.000000,0.038000,6,0,0.038000,0.000,0.006333,0.006333,3.00
3,1043,4,30.866667,30.866667,4.833333,26.033333,4,0,26.033333,0.000,7.716667,6.508333,7.75
4,105051,1,0.002000,0.002000,0.000000,0.002000,0,1,0.000000,0.002,0.002000,0.002000,5.00


In [220]:
df.shape

(1174, 13)

### df3-8 求最大值 最小值

### merge所有中间表结果

In [256]:
df.head()

,customerid,des_cnt,des_charge_sum,des_procharge_sum,des_reduce_sum,des_realcharge_sum,des_workday_cnt,des_weekend_cnt,des_work_realchargesum,des_weekend_realchargesum,des_charge_mean,des_realcharge_mean,des_guestnumber_mean,des_guestnumber_max,des_guestnumber_min,des_charge_max,des_charge_min,des_realcharge_max,des_realcharge_min
0,0,2922,320684.000000,320684.000000,0.000000,320684.000000,1587,1335,187437.000000,133247.000,109.748118,109.748118,1.00,1,1,1127.000000,10.000000,1127.000000,10.000000
1,1008882,2,0.114286,0.114286,0.000000,0.114286,2,0,0.114286,0.000,0.057143,0.057143,4.00,7,1,0.100000,0.014286,0.100000,0.014286
2,104173,6,0.038000,0.036000,0.000000,0.038000,6,0,0.038000,0.000,0.006333,0.006333,3.00,5,1,0.010000,0.002000,0.010000,0.002000
3,1043,4,30.866667,30.866667,4.833333,26.033333,4,0,26.033333,0.000,7.716667,6.508333,7.75,10,5,9.666667,5.600000,8.833333,5.600000
4,105051,1,0.002000,0.002000,0.000000,0.002000,0,1,0.000000,0.002,0.002000,0.002000,5.00,5,5,0.002000,0.002000,0.002000,0.002000


In [257]:
df.shape

(1174, 19)

# 2. 包厢类型统计

In [278]:
df.head()

,customerid,des_large_cnt,des_medium_cnt,des_small_cnt,des_vip_cnt,des_reduce_cnt
0,679446,0,0,0,0,0
1,0,0,0,1,0,0
2,679446,0,0,0,0,0
3,0,0,1,0,0,0
4,0,0,0,1,0,0


In [279]:
pd.value_counts(df['des_large_cnt'])

0    4418
1     161
Name: des_large_cnt, dtype: int64

In [280]:
pd.value_counts(df['des_medium_cnt'])

0    3401
1    1178
Name: des_medium_cnt, dtype: int64

In [281]:
pd.value_counts(df['des_small_cnt'])

0    3423
1    1156
Name: des_small_cnt, dtype: int64

### 弃掉vip数据

In [282]:
pd.value_counts(df['des_vip_cnt'])

0    4579
Name: des_vip_cnt, dtype: int64

In [283]:
pd.value_counts(df['des_reduce_cnt'])

0    4570
1       9
Name: des_reduce_cnt, dtype: int64

### 弃掉vip后的type数据

In [287]:
df.head()

,customerid,des_large_cnt,des_medium_cnt,des_small_cnt,des_reduce_cnt
0,679446,0,0,0,0
1,0,0,0,1,0
2,679446,0,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0


## sum所有单条数据

In [291]:
df.head()

,customerid,des_large_cnt,des_medium_cnt,des_small_cnt,des_reduce_cnt
0,0,139,1105,1073,0
1,1008882,0,0,0,0
2,104173,0,0,0,0
3,1043,0,0,0,0
4,105051,0,0,0,0


In [292]:
pd.value_counts(df['des_large_cnt'])

0      1155
1        14
2         4
139       1
Name: des_large_cnt, dtype: int64

In [293]:
pd.value_counts(df['des_medium_cnt'])

0       1111
1         53
2          7
3          2
1105       1
Name: des_medium_cnt, dtype: int64

In [294]:
pd.value_counts(df['des_small_cnt'])

0       1107
1         54
2          8
3          3
1073       1
4          1
Name: des_small_cnt, dtype: int64

In [295]:
pd.value_counts(df['des_reduce_cnt'])

0    1168
1       5
4       1
Name: des_reduce_cnt, dtype: int64

# 3. 时间相关统计操作

In [67]:
df.head()

,customerid,use_time_sum,des_beforedawn_cnt,des_morning_cnt,des_afternoon_cnt,des_night_cnt,des_use_beforedawn_cnt,des_use_morning_cnt,des_use_afternoon_cnt,des_use_night_cnt,des_use_beforedawn_realchargesum,des_use_morning_realchargesum,des_use_afternoon_realchargesum,des_use_night_realchargesum
0,0,13412,74,498,1223,1127,66,0,1657,1199,8761,0,71568.000000,240355.000000
1,1008882,1,0,0,1,1,0,0,1,1,0,0,0.100000,0.014286
2,104173,3,0,0,4,2,0,0,4,2,0,0,0.034000,0.004000
3,1043,23,0,2,1,1,0,0,4,0,0,0,26.033333,0.000000
4,105051,1,0,1,0,0,0,0,0,1,0,0,0.000000,0.002000


In [68]:
df.shape

(1174, 14)

### 求用户预订-使用包厢平均时间

In [127]:
df.head()

,customerid,des_use_reg
0,0,0.161191
1,1008882,0.000000
2,104173,-0.166667
3,1043,0.750000
4,105051,0.000000


In [128]:
df.shape

(1174, 2)

### 表merge后结果

In [122]:
df.head()

,customerid,use_time_sum,des_beforedawn_cnt,des_morning_cnt,des_afternoon_cnt,des_night_cnt,des_use_beforedawn_cnt,des_use_morning_cnt,des_use_afternoon_cnt,des_use_night_cnt,des_use_beforedawn_realchargesum,des_use_morning_realchargesum,des_use_afternoon_realchargesum,des_use_night_realchargesum,des_use_reg
0,0,13412,74,498,1223,1127,66,0,1657,1199,8761,0,71568.000000,240355.000000,0.161191
1,1008882,1,0,0,1,1,0,0,1,1,0,0,0.100000,0.014286,0.000000
2,104173,3,0,0,4,2,0,0,4,2,0,0,0.034000,0.004000,-0.166667
3,1043,23,0,2,1,1,0,0,4,0,0,0,26.033333,0.000000,0.750000
4,105051,1,0,1,0,0,0,0,0,1,0,0,0.000000,0.002000,0.000000


In [123]:
df.shape

(1174, 15)

## 3. 统计消费所在商户数据

### 保留用户消费次数最多的商户记录

In [144]:
df.head()

,customerid,des_companyid,cnt
26,0,8158,292
28,1008882,10433,1
30,104173,742,6
31,1043,4018,2
34,105051,742,1


In [145]:
df.shape

(1174, 3)

## 消费商户场所静态数据

In [149]:
df.head()

,des_companyid,des_com_roomnum,des_com_managetype,des_com_vodsystemtype,des_com_regioncode
0,136,89,1,0,500000
1,139,136,1,0,440000
2,147,70,1,0,350000
3,168,67,1,0,440000
4,180,55,1,0,350000


In [150]:
df.shape

(936, 5)

## merge所有表

### merge des_com,company

In [154]:
df.head()

,customerid,des_companyid,cnt,des_com_roomnum,des_com_managetype,des_com_vodsystemtype,des_com_regioncode
0,0,8158,292,72,1,0,500000
1,1008882,10433,1,NaN,NaN,NaN,NaN
2,104173,742,6,NaN,NaN,NaN,NaN
3,1043,4018,2,NaN,NaN,NaN,NaN
4,105051,742,1,NaN,NaN,NaN,NaN


### 加上des_time

In [7]:
df.head()

,customerid,use_time_sum,des_beforedawn_cnt,des_morning_cnt,des_afternoon_cnt,des_night_cnt,des_use_beforedawn_cnt,des_use_morning_cnt,des_use_afternoon_cnt,des_use_night_cnt,...,des_use_morning_realchargesum,des_use_afternoon_realchargesum,des_use_night_realchargesum,des_use_reg,des_companyid,cnt,des_com_roomnum,des_com_managetype,des_com_vodsystemtype,des_com_regioncode
0,0,13412,74,498,1223,1127,66,0,1657,1199,...,0,71568.000000,240355.000000,0.161191,8158,292,72,1,0,500000
1,1008882,1,0,0,1,1,0,0,1,1,...,0,0.100000,0.014286,0.000000,10433,1,NaN,NaN,NaN,NaN
2,104173,3,0,0,4,2,0,0,4,2,...,0,0.034000,0.004000,-0.166667,742,6,NaN,NaN,NaN,NaN
3,1043,23,0,2,1,1,0,0,4,0,...,0,26.033333,0.000000,0.750000,4018,2,NaN,NaN,NaN,NaN
4,105051,1,0,1,0,0,0,0,0,1,...,0,0.000000,0.002000,0.000000,742,1,NaN,NaN,NaN,NaN


In [8]:
df.shape

(1174, 21)

In [9]:
df.columns

Index([u'customerid', u'use_time_sum', u'des_beforedawn_cnt',
       u'des_morning_cnt', u'des_afternoon_cnt', u'des_night_cnt',
       u'des_use_beforedawn_cnt', u'des_use_morning_cnt',
       u'des_use_afternoon_cnt', u'des_use_night_cnt',
       u'des_use_beforedawn_realchargesum', u'des_use_morning_realchargesum',
       u'des_use_afternoon_realchargesum', u'des_use_night_realchargesum',
       u'des_use_reg', u'des_companyid', u'cnt', u'des_com_roomnum',
       u'des_com_managetype', u'des_com_vodsystemtype', u'des_com_regioncode'],
      dtype='object')

### 加上des_type

In [14]:
df.head()

,customerid,use_time_sum,des_beforedawn_cnt,des_morning_cnt,des_afternoon_cnt,des_night_cnt,des_use_beforedawn_cnt,des_use_morning_cnt,des_use_afternoon_cnt,des_use_night_cnt,...,des_companyid,cnt,des_com_roomnum,des_com_managetype,des_com_vodsystemtype,des_com_regioncode,des_large_cnt,des_medium_cnt,des_small_cnt,des_reduce_cnt
0,0,13412,74,498,1223,1127,66,0,1657,1199,...,8158,292,72,1,0,500000,139,1105,1073,0
1,1008882,1,0,0,1,1,0,0,1,1,...,10433,1,NaN,NaN,NaN,NaN,0,0,0,0
2,104173,3,0,0,4,2,0,0,4,2,...,742,6,NaN,NaN,NaN,NaN,0,0,0,0
3,1043,23,0,2,1,1,0,0,4,0,...,4018,2,NaN,NaN,NaN,NaN,0,0,0,0
4,105051,1,0,1,0,0,0,0,0,1,...,742,1,NaN,NaN,NaN,NaN,0,0,0,0


In [15]:
df.shape

(1174, 25)

In [16]:
df.columns

Index([u'customerid', u'use_time_sum', u'des_beforedawn_cnt',
       u'des_morning_cnt', u'des_afternoon_cnt', u'des_night_cnt',
       u'des_use_beforedawn_cnt', u'des_use_morning_cnt',
       u'des_use_afternoon_cnt', u'des_use_night_cnt',
       u'des_use_beforedawn_realchargesum', u'des_use_morning_realchargesum',
       u'des_use_afternoon_realchargesum', u'des_use_night_realchargesum',
       u'des_use_reg', u'des_companyid', u'cnt', u'des_com_roomnum',
       u'des_com_managetype', u'des_com_vodsystemtype', u'des_com_regioncode',
       u'des_large_cnt', u'des_medium_cnt', u'des_small_cnt',
       u'des_reduce_cnt'],
      dtype='object')

### 加上des_stat

In [21]:
df.head()

,customerid,des_cnt,des_charge_sum,des_procharge_sum,des_reduce_sum,des_realcharge_sum,des_workday_cnt,des_weekend_cnt,des_work_realchargesum,des_weekend_realchargesum,...,des_companyid,cnt,des_com_roomnum,des_com_managetype,des_com_vodsystemtype,des_com_regioncode,des_large_cnt,des_medium_cnt,des_small_cnt,des_reduce_cnt
0,0,2922,320684.000000,320684.000000,0.000000,320684.000000,1587,1335,187437.000000,133247.000,...,8158,292,72,1,0,500000,139,1105,1073,0
1,1008882,2,0.114286,0.114286,0.000000,0.114286,2,0,0.114286,0.000,...,10433,1,NaN,NaN,NaN,NaN,0,0,0,0
2,104173,6,0.038000,0.036000,0.000000,0.038000,6,0,0.038000,0.000,...,742,6,NaN,NaN,NaN,NaN,0,0,0,0
3,1043,4,30.866667,30.866667,4.833333,26.033333,4,0,26.033333,0.000,...,4018,2,NaN,NaN,NaN,NaN,0,0,0,0
4,105051,1,0.002000,0.002000,0.000000,0.002000,0,1,0.000000,0.002,...,742,1,NaN,NaN,NaN,NaN,0,0,0,0


In [22]:
df.shape

(1174, 43)

In [23]:
df.columns

Index([u'customerid', u'des_cnt', u'des_charge_sum', u'des_procharge_sum',
       u'des_reduce_sum', u'des_realcharge_sum', u'des_workday_cnt',
       u'des_weekend_cnt', u'des_work_realchargesum',
       u'des_weekend_realchargesum', u'des_charge_mean',
       u'des_realcharge_mean', u'des_guestnumber_mean', u'des_guestnumber_max',
       u'des_guestnumber_min', u'des_charge_max', u'des_charge_min',
       u'des_realcharge_max', u'des_realcharge_min', u'use_time_sum',
       u'des_beforedawn_cnt', u'des_morning_cnt', u'des_afternoon_cnt',
       u'des_night_cnt', u'des_use_beforedawn_cnt', u'des_use_morning_cnt',
       u'des_use_afternoon_cnt', u'des_use_night_cnt',
       u'des_use_beforedawn_realchargesum', u'des_use_morning_realchargesum',
       u'des_use_afternoon_realchargesum', u'des_use_night_realchargesum',
       u'des_use_reg', u'des_companyid', u'cnt', u'des_com_roomnum',
       u'des_com_managetype', u'des_com_vodsystemtype', u'des_com_regioncode',
       u'des_large_cnt